## Production Dashboarding ##
panel serve PHNX_prod_analytics.ipynb --session-token-expiration 600 <br>
kill -9 $(lsof -ti:5006) <br>
* https://docs.google.com/document/d/1PU1OUlR6i1fGRnLsv8iYXmmY_byGu2kccWzCMyY922o/edit

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import panel as pn
import pickle
import numpy as np
cwd = os.getcwd()
slash_idx = [i for i,l in enumerate(cwd) if l=='/']
repo_dir = cwd[:slash_idx[-2]]
sys.path.append(repo_dir)
from analysis.analysis_pipeline import AnalysisPipeline
from utils.lims_utils import get_plate_runs, plates_from_exp, plates_from_workflow
from utils.s3_interface import download_from_s3, upload_to_s3, s3_imgupload, s3_df2csv, s3_csv2df
from utils.table_properties import * 
pd.set_option('display.max_columns', None)
s3_model_bucket = 'ml-analytics-file-store'
s3_model_bucket_subdirectory = 'exerevnetes-preprocessing-models/'

pn.extension(sizing_mode="stretch_width")
overall_analytics_panel = []
variant_analytics_panel = []
top_variant_reps = []
top_variant_analytics_panel = []
figure_panel = []
plate_analytics_panel = []
library_analytics_panel = []
unit_analytics_panel = []

top_variant_stats, top_variant_reps = None, None

# Get data from table

In [3]:
project_subfolder = 'Phoenix-prod_Unit2-3-4-5-6-7-8-9-10-11-12-13-14-15-16_LCMSONLY/' 
exp_workflow = ['WF10125','WF10128','WF10130','WF10132', 'WF10134', 'WF10136', 'WF10139', 'WF10144', 'WF10149', 'WF10157', 'WF10158', 'WF10163', 'WF10169', 'WF10173', 'WF10181']  # None # 
table = 'combi_analytics_table' 
metric_dict = {'LcmsC18':['pellet_OD', 'measured_nonbinary_sum_(r)','measured_nonbinary_score_(r)', 'measured_conversion_(r)'], 'LcmsChiral':['measured_enantiomeric_excess_(+over-)']}
analysis_pipeline = AnalysisPipeline(s3_subfolder=project_subfolder, neg_ctrltype='EV', sort_by=None, metric_dict=metric_dict, get_dashboard_panel=False)
df = analysis_pipeline.load_data(table, exp_workflow=exp_workflow)

executing query: SELECT "id", "exp_workflow_barcode", "exp_workflow_name", "proj_barcode", "proj_name", "maldi_run", "lcms_C18_run", "lcms_chiral_run", "maldi_plate_(r)", "maldi_address_(r)", "lcms_plate_(r)", "lcms_address_(r)", "source_plate_(r)", "source_address_(r)", "maldi_plate_(+)", "maldi_address_(+)", "lcms_plate_(+)", "lcms_address_(+)", "source_plate_(+)", "source_address_(+)", "maldi_plate_(-)", "maldi_address_(-)", "lcms_plate_(-)", "lcms_address_(-)", "source_plate_(-)", "source_address_(-)", "ctrl_type", "exp_condition", "enzyme_barcode", "sequence", "mutations", "hamming", "reference_enzyme", "substrate_barcode_(r)", "substrate_barcode_(+)", "substrate_barcode_(-)", "substrate_smiles_(r)", "substrate_smiles_(+)", "substrate_smiles_(-)", "substrate_concentration_(r)", "substrate_concentration_(+)", "substrate_concentration_(-)", "predicted_binary_score_(r)", "predicted_nonbinary_score_(r)", "measured_nonbinary_score_(r)", "measured_nonbinary_sum_(r)", "measured_conversio

# Get Analytics from data

In [ ]:
# overall_analytics, overall_analytics_panel = analysis_pipeline.GET_OVERALL_ANALYTICS(df, display_table=True, plot_scatterplot=True, plot_histogram=True)
# variant_analytics, variant_analytics_panel = analysis_pipeline.GET_VARIANT_ANALYTICS(df, display_table=True, plot_histogram=True)
# top_variant_stats, top_variant_reps, top_variant_analytics_panel = analysis_pipeline.get_top_variants(
#     variant_analytics, df, 
#     thres_dict={'measured_conversion_(r)':(0.2,1,'median'), 'measured_enantiomeric_excess_(+over-)':(0.2,1,'median')}, 
#     top_n_variants=100, sort_by=('measured_conversion_(r)', 'median'), 
#     get_replicates_data=True
# )
library_analytics, library_analytics_panel = analysis_pipeline.GET_LIBRARY_ANALYTICS(df, display_table=True, plot_histogram=True)
plate_analytics, plate_analytics_panel = analysis_pipeline.GET_PLATE_ANALYTICS(df, display_table=True)
unit_analytics, unit_analytics_panel = analysis_pipeline.GET_UNIT_ANALYTICS(df, display_table=True, plot_scatterplot=True, plot_histogram=True, get_ctrls_from_vals=True)

## Get additional visualizations

In [ ]:
metric_list = ['measured_nonbinary_score_(r)', 'pellet_OD', 'measured_nonbinary_sum_(r)']
metricname_list=['RacemicProduct', 'PelletOD', 'RacemicSum']

if top_variant_reps is not None and top_variant_stats is not None:
    # get boxplots for activity for each variant
    boxplot_activity_corr_list = analysis_pipeline.get_activity_correlation_boxplots(top_variant_reps, metric_list=metric_list, metricname_list=metricname_list, groupby=['mutations'], table_suffix='\n(all var)')

    scatterplot_activity_corr_lists = {}
    for metric, metricname in zip(metric_list[1:], metricname_list[1:]):

        # get scatter plot of activity for all variants
        scatterplot_activity_corr_list_ALL = analysis_pipeline.get_activity_correlation_scatterplots(top_variant_reps, xmetric_list=[metric], xmetricname_list=[metricname], ymetric=metric_list[0], ymetricname=metricname_list[0], groupby='mutations', table_suffix='\n(all var)')

        # get scatter plots for each variant
        scatterplot_activity_corr_list_VARS = []
        var_to_plot = top_variant_stats.sort_values(by=['n_LcmsC18', 'measured_conversion_(r)_median'], ascending=False).mutations.tolist()[:10]
        for var in var_to_plot:
            var_reps = top_variant_reps.loc[top_variant_reps.mutations==var]
            scatterplot_activity_corr_list_VAR = analysis_pipeline.get_activity_correlation_scatterplots(var_reps, xmetric_list=[metric], xmetricname_list=[metricname], ymetric=metric_list[0], ymetricname=metricname_list[0], groupby='ctrl_type', table_suffix=f'\n[{var}]')
            scatterplot_activity_corr_list_VARS += scatterplot_activity_corr_list_VAR

        # update fig list
        scatterplot_activity_corr_lists[metricname] = scatterplot_activity_corr_list_ALL + scatterplot_activity_corr_list_VARS

    # get scatter plots of activity CV for all variants
    scatterplot_activityCV_corr_list = analysis_pipeline.get_activity_CV_correlation_scatterplots(top_variant_stats, xmetric_list=['pellet_OD_cv', 'measured_nonbinary_sum_(r)_cv'], xmetricname_list=['PelletOD-CV', 'RacemicSum-CV'], ymetric='measured_nonbinary_score_(r)_cv', ymetricname='RacemicProduct-CV', groupby='mutations', table_suffix='\n(all var)')

# Get Analytics Dashboard

In [140]:
# figure panel
if top_variant_reps is not None and top_variant_stats is not None:
    activity_boxplots = pn.Column(
        '# Boxplots of Activity, PelletOD & SumConc (by variant)', 
        *[pn.pane.Matplotlib(fig) for fig in boxplot_activity_corr_list], 
        background='White', scroll=True)
    
    gspec_dict = {metric: pn.GridSpec(sizing_mode='stretch_both', max_height=600) for metric in scatterplot_activity_corr_lists}
    n_plots = len(list(scatterplot_activity_corr_lists.values())[0])
    rows = 2
    cols = int(np.ceil(n_plots/2))
    count = 0
    while count < n_plots:
        i = int(np.floor(count/cols))
        j = int(count%cols)
        for metric in scatterplot_activity_corr_lists:
            gspec_dict[metric][i, j] = pn.pane.Matplotlib(scatterplot_activity_corr_lists[metric][count])
        count += 1
    
    activity_scatterplots = pn.Column(
        '# Scatter plots of Activity VS. PelletOD (by sample)', 
        gspec_dict['PelletOD'], 
        '# Scatter plots of Activity VS. SumConc (by sample)', 
        gspec_dict['RacemicSum'], 
        background='White', scroll=True)

    activityCV_scatterplots = pn.Column(
        '# Scatter plots of Activity CV VS. PelletOD CV & SumConc CV (by variant)', 
        pn.Row(*[pn.pane.Matplotlib(fig) for fig in scatterplot_activityCV_corr_list]), 
        background='White', scroll=True)

    figure_panel = [activity_boxplots, activity_scatterplots, activityCV_scatterplots]

In [ ]:
analytics_panel = overall_analytics_panel + variant_analytics_panel + top_variant_analytics_panel + top_variant_analytics_panel + figure_panel + plate_analytics_panel + library_analytics_panel + unit_analytics_panel
# analytics_panel = figure_panel
pn.template.FastListTemplate(
    site="Panel", 
    title="PHNX Production Analytics", 
    main=analytics_panel, 
    main_max_width="1760px",
).servable();

# Additional Manual Analysis

## Plot control stats

In [ ]:
from utils.visualization_utils import plot_boxplot
from utils.analysis_utils import calculate_stats_for_dataframe

metric_list=['measured_nonbinary_score_(r)', 'measured_conversion_(r)', 'measured_enantiomeric_excess_(+over-)', 'pellet_OD', 'measured_nonbinary_sum_(r)']
metricname_list=['RacemicProduct', 'RacemicConversion', 'ChiralEE', 'PelletOD', 'RacemicSum']
groupby_list = ['exp_workflow_barcode', 'library_ref', 'lcms_plate_(r)']

ctrl_data = {}
df.loc[df['substrate_concentration_(r)']>300, 'substrate_concentration_(r)'] = 308
ctrl_data['pos'] = df.loc[df.ctrl_type.isin(['pos'])]
ctrl_data['EV']  = df.loc[df.ctrl_type.isin(['EV'])]

for ctrl_type in ['pos', 'EV']:
    
    for i, (metric, metricname) in enumerate(zip(metric_list, metricname_list)):
        data = ctrl_data[ctrl_type]
        print(ctrl_type, metric)
        
        # # get overall boxplots
        # fig, ax = plt.subplots(figsize=(6,6))
        # data.boxplot(column=metric, ax=ax, by='substrate_concentration_(r)')
        # ax.set_title(f'[{ctrl_type.upper()}] Boxplot of {metric}')
        # plt.show(fig)
        
        # # get overall histograms
        # fig, ax = plt.subplots(1,2, figsize=(12,6))
        # data.hist(column=metric, ax=ax, bins=int(len(data)/15), by='substrate_concentration_(r)')
        # plt.suptitle(f'[{ctrl_type.upper()}] Histogram of {metric}')
        # ax[0].set_yscale('log')
        # ax[1].set_yscale('log')
        # plt.show(fig)        
        
        
        for sub_conc in [140, 308]:
            
            # # get overall stats
            # print('OVERALL STATS')
            # data_subconc = data.loc[data['substrate_concentration_(r)']==sub_conc, [metric]]
            # metrics_dict = calculate_stats_for_dataframe(df=data_subconc, colname=metric, metric_prefix=f'{metric}_', stats_to_get=['mean', 'median', 'std','iqr','cv'], metrics_dict={})
            # print(pd.DataFrame.from_dict(metrics_dict, orient='index', columns=[sub_conc]).round(4))
            
#             # get unit stats
#             # print('UNIT STATS')
#             metric_stat = f'{metric}_mean_{ctrl_type.upper()}'
#             unit_analytics_subconc = unit_analytics.iloc[3:].loc[unit_analytics.iloc[3:]['substrate_concentration_(r)']==sub_conc, [metric_stat]]
#             metrics_dict = calculate_stats_for_dataframe(df=unit_analytics_subconc, colname=metric_stat, metric_prefix=f'{metric_stat}_', stats_to_get=['mean', 'median', 'std','iqr','cv'], metrics_dict={})
#             print(pd.DataFrame.from_dict(metrics_dict, orient='index', columns=[sub_conc]).round(4))
            
#             metric_stat = f'{metric}_std_{ctrl_type.upper()}'
#             unit_analytics_subconc = unit_analytics.iloc[3:].loc[unit_analytics.iloc[3:]['substrate_concentration_(r)']==sub_conc, [metric_stat]]
#             metrics_dict = calculate_stats_for_dataframe(df=unit_analytics_subconc, colname=metric_stat, metric_prefix=f'{metric_stat}_', stats_to_get=['mean', 'median', 'std','iqr','cv'], metrics_dict={})
#             print(pd.DataFrame.from_dict(metrics_dict, orient='index', columns=[sub_conc]).round(4))
            
            # get plate stats
            metric_stat = f'{metric}_mean_{ctrl_type.upper()}'
            plate_analytics_subconc = plate_analytics.iloc[3:].loc[plate_analytics.iloc[3:]['substrate_concentration_(r)']==sub_conc, [metric_stat]]
            metrics_dict = calculate_stats_for_dataframe(df=plate_analytics_subconc, colname=metric_stat, metric_prefix=f'{metric_stat}_', stats_to_get=['mean', 'median', 'std','iqr','cv'], metrics_dict={})
            print(pd.DataFrame.from_dict(metrics_dict, orient='index', columns=[sub_conc]).round(4))
            
            metric_stat = f'{metric}_std_{ctrl_type.upper()}'
            plate_analytics_subconc = unit_analytics.iloc[3:].loc[plate_analytics.iloc[3:]['substrate_concentration_(r)']==sub_conc, [metric_stat]]
            metrics_dict = calculate_stats_for_dataframe(df=plate_analytics_subconc, colname=metric_stat, metric_prefix=f'{metric_stat}_', stats_to_get=['mean', 'median', 'std','iqr','cv'], metrics_dict={})
            print(pd.DataFrame.from_dict(metrics_dict, orient='index', columns=[sub_conc]).round(4))
                    
            
#         # get grouped boxplots
#         for groupby in groupby_list: 

#             # get grouped histograms
#             grp_list = list(set(data[groupby]))
#             fig, ax = plt.subplots(figsize=(10,10))
#             data.hist(column=metric, ax=ax, bins=int(len(data)/30), by=groupby)
#             ax.set_title(f'[{ctrl_type.upper()}] Histogram of {metric} grouped by {groupby}')
#             plt.show(fig)  
            
#             plot_boxplot(data, [metric], metricnames=[metricname], groupby=[groupby], fname_prefix=f'{ctrl_type.upper()}_', img_format='png', 
#                          save_plot_to_s3=True, s3_bucket='ml-analytics-file-store', s3_subfolder=project_subfolder, showplot=True, show_n=True)

## Plot variant stats

In [211]:
# metric_list=[
#     'measured_nonbinary_score_(r)_mean', 'measured_nonbinary_score_(r)_std', 'measured_nonbinary_score_(r)_iqr', 'measured_nonbinary_score_(r)_cv', 
#     'measured_conversion_(r)_mean', 'measured_conversion_(r)_std', 'measured_conversion_(r)_iqr', 'measured_conversion_(r)_cv', 
#     'measured_enantiomeric_excess_(+over-)_mean', 'measured_enantiomeric_excess_(+over-)_std', 'measured_enantiomeric_excess_(+over-)_iqr', 'measured_enantiomeric_excess_(+over-)_cv',
#     'pellet_OD_mean', 'pellet_OD_median', 'pellet_OD_std', 'pellet_OD_iqr', 'pellet_OD_cv', 
#     'measured_nonbinary_sum_(r)_mean', 'measured_nonbinary_sum_(r)_median', 'measured_nonbinary_sum_(r)_std', 'measured_nonbinary_sum_(r)_iqr', 'measured_nonbinary_sum_(r)_cv', 
#             ]

# metricname_list=[
#     'RacemicProduct-MEAN', 'RacemicProduct-STDEV', 'RacemicProduct-IQR', 'RacemicProduct-CV', 
#     'RacemicConversion-MEAN', 'RacemicConversion-STDEV', 'RacemicConversion-IQR', 'RacemicConversion-CV', 
#     'ChiralEE-MEAN', 'ChiralEE-STDEV', 'ChiralEE-IQR', 'ChiralEE-CV', 
#     'PelletOD-MEAN', 'PelletOD-MEDIAN', 'PelletOD-STDEV', 'PelletOD-IQR', 'PelletOD-CV', 
#     'RacemicSum-MEAN', 'RacemicSum-MEDIAN', 'RacemicSum-STDEV', 'RacemicSum-IQR', 'RacemicSum-CV', 
#                 ] 

metric_list=[
    'measured_nonbinary_score_(r)_std', 'measured_nonbinary_score_(r)_cv',
    'measured_conversion_(r)_std', 'measured_conversion_(r)_cv', 
    'measured_enantiomeric_excess_(+over-)_std', 'measured_enantiomeric_excess_(+over-)_cv', 
    'pellet_OD_mean', 'pellet_OD_std', 
    'measured_nonbinary_sum_(r)_mean', 'measured_nonbinary_sum_(r)_std', 
            ]

metricname_list=[
    'RacemicProduct-STDEV', 'RacemicProduct-CV', 
    'RacemicConversion-STDEV', 'RacemicConversion-CV', 
    'ChiralEE-STDEV', 'ChiralEE-CV', 
    'PelletOD-MEAN', 'PelletOD-STDEV', 
    'RacemicSum-MEAN', 'RacemicSum-STDEV', 
                ] 

groupby_list = ['exp_workflow_barcode', 'library_ref', 'lcms_plate_(r)']

variant_analytics_noEV = variant_analytics.iloc[3:].copy()
variant_analytics_noEV = variant_analytics_noEV.loc[~variant_analytics_noEV.enzyme_barcode.isin(['ENZ10045', 'DNA10001'])]
variant_analytics_noEV.loc[variant_analytics_noEV['substrate_concentration_(r)']>300, 'substrate_concentration_(r)'] = 308
variant_analytics_noEV = variant_analytics_noEV.loc[variant_analytics_noEV['n_LcmsC18'] >= 4]
    
for i, (metric, metricname) in enumerate(zip(metric_list, metricname_list)):
    
    for sub_conc in [140, 308]:
        variant_analytics_noEV_subconc = variant_analytics_noEV.loc[variant_analytics_noEV['substrate_concentration_(r)']==sub_conc]
        # get OVERALL stats
#         stats_to_get = ['mean', 'median', 'std', 'iqr', 'cv']
#         metrics_dict = calculate_stats_for_dataframe(df=variant_analytics_noEV_subconc, colname=metric, metric_prefix=f'{metric}_', stats_to_get=stats_to_get, metrics_dict={})
#         print(pd.DataFrame.from_dict(metrics_dict, orient='index', columns=[sub_conc]).round(4))

#         # get UNIT stats
#         print('UNIT STATS')        
#         # get groups
#         grp_list = sorted(list(set(variant_analytics_noEV_subconc['exp_workflow_barcode'])))
#         grp_len = len(grp_list)
 
#         # sort dataframe into groups
#         df_grps = {}
#         for grp in grp_list: 
#             df_grps[grp] = variant_analytics_noEV_subconc.loc[variant_analytics_noEV_subconc['exp_workflow_barcode']==grp, metric]
#         df_grps = pd.DataFrame(df_grps)
#         df_grps_mean = pd.DataFrame(df_grps.mean(), columns=[metric])
        
#         # get stats
#         metrics_dict = calculate_stats_for_dataframe(df=df_grps_mean, colname=metric, metric_prefix=f'{metric}_', stats_to_get=['mean', 'median', 'std','iqr','cv'], metrics_dict={})
#         print(pd.DataFrame.from_dict(metrics_dict, orient='index', columns=[sub_conc]).round(4))

        # get PLATE stats      
        # get groups
        grp_list = sorted(list(set(variant_analytics_noEV_subconc['lcms_plate_(r)'])))
        grp_len = len(grp_list)
 
        # sort dataframe into groups
        df_grps = {}
        for grp in grp_list: 
            df_grps[grp] = variant_analytics_noEV_subconc.loc[variant_analytics_noEV_subconc['lcms_plate_(r)']==grp, metric]
        df_grps = pd.DataFrame(df_grps)
        df_grps_mean = pd.DataFrame(df_grps.mean(), columns=[metric])
        
        # get stats
        metrics_dict = calculate_stats_for_dataframe(df=df_grps_mean, colname=metric, metric_prefix=f'{metric}_', stats_to_get=['mean', 'median', 'std','iqr','cv'], metrics_dict={})
        print(pd.DataFrame.from_dict(metrics_dict, orient='index', columns=[sub_conc]).round(4))

#     # get overall boxplots
#     fig, ax = plt.subplots(figsize=(6,6))
#     variant_analytics_noEV.boxplot(column=metric, ax=ax, by='substrate_concentration_(r)')
#     ax.set_title(f'[VAR-ONLY] Boxplot of {metric}')
#     plt.show(fig)
    
#     # get overall histograms
#     fig, ax = plt.subplots(1,2, figsize=(12,6))
#     variant_analytics_noEV.hist(column=metric, ax=ax, bins=int(len(variant_analytics_noEV)/10), by='substrate_concentration_(r)')
#     ax[0].set_yscale('log')
#     ax[1].set_yscale('log')
#     plt.suptitle(f'[VAR ONLY] Histogram of {metric}')
#     plt.show(fig)  

#     for groupby in groupby_list:
        
#         grp_list = list(set(variant_analytics_noEV[groupby]))
#         for grp in grp_list: 
#             variant_analytics_noEV.loc[variant_analytics_noEV[groupby]==grp, metric]
            
#         variant_analytics_noEV = variant_analytics_noEV.sort_values(by=groupby)
#         plot_boxplot(variant_analytics_noEV, [metric], metricnames=[metricname], groupby=[groupby], fname_prefix=f'VAR-ONLY_', img_format='png', 
#                      save_plot_to_s3=True, s3_bucket='ml-analytics-file-store', s3_subfolder=project_subfolder, showplot=True, show_n=True)        

                                          140
measured_nonbinary_score_(r)_std_mean   1.855
measured_nonbinary_score_(r)_std_median 0.809
measured_nonbinary_score_(r)_std_std    3.368
measured_nonbinary_score_(r)_std_iqr    1.491
measured_nonbinary_score_(r)_std_cv     1.815
                                           308
measured_nonbinary_score_(r)_std_mean    8.009
measured_nonbinary_score_(r)_std_median  3.866
measured_nonbinary_score_(r)_std_std    14.915
measured_nonbinary_score_(r)_std_iqr     8.777
measured_nonbinary_score_(r)_std_cv      1.862
                                         140
measured_nonbinary_score_(r)_cv_mean   0.753
measured_nonbinary_score_(r)_cv_median 0.390
measured_nonbinary_score_(r)_cv_std    1.312
measured_nonbinary_score_(r)_cv_iqr    0.461
measured_nonbinary_score_(r)_cv_cv     1.742
                                         308
measured_nonbinary_score_(r)_cv_mean   0.874
measured_nonbinary_score_(r)_cv_median 0.178
measured_nonbinary_score_(r)_cv_std  